Case: Multi‑period Real‑Estate Co‑Investment

A developer can buy fractional shares of three projects—an office tower (O), a hotel (H), and a shopping center (S). Buying a share of any project commits the firm to fund four staged investments (now, year 1, year 2, year 3) in proportion to its share. Each project also has a total net present value (NPV) for a 100% stake; a partial stake earns that same fraction of NPV.

The firm has separate capital budgets available at each time point (now: $25M; year 1: $20M; year 2: $20M; year 3: $15M). Unused funds at one time do not carry forward.

Investment requirements per project (per 100% stake) and total NPV:

Project	Now	Y1	Y2	Y3	NPV
Office (O)	40	60	90	10	45
Hotel (H)	80	80	80	70	70
Shopping (S)	90	50	20	60	50

(All values in $ millions.)

Goal: Choose shares to maximize total NPV, subject to the per‑period capital limits.